In [3]:
import numpy as np 
import faiss  # faiss 라이브러리 호출

In [4]:
dimension = 128    # 각 벡터의 차원                
n = 200    # 벡터 수                   
np.random.seed(1)             
db_vectors = np.random.random((n, dimension)).astype('float32') #(200 * 128) 벡터 행렬
db_vectors

array([[4.17021990e-01, 7.20324516e-01, 1.14374816e-04, ...,
        5.38831055e-01, 5.52821994e-01, 8.42030883e-01],
       [1.24173313e-01, 2.79183686e-01, 5.85759282e-01, ...,
        9.73783553e-01, 6.04716122e-01, 8.28845799e-01],
       [5.74711502e-01, 6.28076196e-01, 2.85576284e-01, ...,
        5.81577420e-01, 9.89751697e-01, 2.03906223e-01],
       ...,
       [5.70789576e-01, 9.44105268e-01, 3.55858296e-01, ...,
        2.26064011e-01, 9.37489152e-01, 7.52344429e-01],
       [3.34620804e-01, 4.76446241e-01, 2.63767540e-01, ...,
        4.18909609e-01, 3.09424341e-01, 7.24189103e-01],
       [5.95713854e-01, 5.31086147e-01, 4.29194756e-02, ...,
        7.86913514e-01, 6.50519311e-01, 1.31239966e-01]], dtype=float32)

In [5]:
nlist = 5  # 클러스터(셀이라고도 표현) 수
quantiser = faiss.IndexFlatL2(dimension)  #quantizer를 활용해서 클러스터 생성
index = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)  #Inverted File 만들기
#index = faiss.IndexPQ(D, m, nbits)
#index = faiss.IndexIVFPQ(vecs, D, nlist, m, nbits)

인덱스를 기반으로 훈련을 시켜야함
- index를 만들어서 쓰는게 FAISS에서 제공이 되고 있음

문제는 이런식으로 index를 이렇게 만들어서 쓰는게, 
- 기본적으로 제공되는 것 대비 속도가 좋다는 것을 아무도 보장할 수 없음
- 많은 노하우가 필요함

In [6]:
print(index.is_trained)   # False
index.train(db_vectors)  # 데이터베이스 벡터에 대한 훈련
print(index.ntotal)

False
0


In [7]:
index.add(db_vectors)   # 벡터를 추가하고 인덱스를 업데이트
print(index.is_trained)  # True
print(index.ntotal) 

True
200


기본적으로 FAISS가 built-in 되어 있는 similarity search를 쓰는게 아니라,
- (langchain에서 제공하는 FAISS에서 제공하는 similarity search)
- index를 우리가 만들었고, 그 index에서 검색해 오도록 설정을 했음

In [8]:
#nprobe = 2  # 가장 유사한 클러스터 2개 찾기
n_query = 10  # 10개의 쿼리 벡터
k = 3  # 가장 가까운 이웃 3개를 반환
np.random.seed(1)  # 일관된 결과를 얻기 위해 시드 설정 
query_vectors = np.random.random((n_query, dimension)).astype('float32') #쿼리 벡터 생성
distances, indices = index.search(query_vectors, k) #검색 수행

In [9]:
distances

array([[ 0.      , 17.15714 , 18.844372],
       [ 0.      , 17.987484, 18.616207],
       [ 0.      , 15.770564, 16.0127  ],
       [ 0.      , 16.52227 , 16.846798],
       [ 0.      , 18.070686, 18.418453],
       [ 0.      , 16.012554, 16.118513],
       [ 0.      , 14.298641, 15.909523],
       [ 0.      , 14.296753, 16.200497],
       [ 0.      , 16.161392, 16.6162  ],
       [ 0.      , 16.274105, 17.641554]], dtype=float32)

In [10]:
indices

array([[  0,  93,  20],
       [  1, 159, 144],
       [  2, 173,  84],
       [  3,   6,  84],
       [  4, 118,  51],
       [  5,  98,  47],
       [  6, 185,  52],
       [  7,  42, 165],
       [  8,  55,  82],
       [  9,  14,  83]])

사실 이 과정들이 쉬운게 아님
- index 자체도 어떤 것을 쓸지 많이 고민해야함 
- train과정에서도 H/W가 많이 사용됨

In [ ]:
faiss.write_index(index, "vector.index")  # 인덱스를 디스크에 저장
index = faiss.read_index("vector.index")  # 인덱스를 로드